In [1]:
from bs4 import BeautifulSoup
import requests
import csv

In [2]:
url = "https://www.storia.ro/ro/rezultate/vanzare/apartament/iasi/iasi"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)

print(response.status_code)

200


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

In [7]:
BASE_URL = "https://www.storia.ro/ro/rezultate/vanzare/apartament/iasi/iasi"
NUMAR_PAGINI = 120
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9,ro;q=0.8"
}


In [9]:
data_colectata = []

for pagina in range(1, NUMAR_PAGINI + 1):
    url_curent = f"{BASE_URL}?page={pagina}"
    print(f"Pagina {pagina}: {url_curent} ...")
    
    try:
        response = requests.get(url_curent, headers=HEADERS)
        
        if response.status_code != 200:
            print(f"Eroare la pagina {pagina} (Cod: {response.status_code})")
            continue
            
        soup = BeautifulSoup(response.content, 'html.parser')
        
        anunturi = soup.find_all('article')
        
        if not anunturi:
            print("Nu s au mai gasit anunturi.")
            break
            
        for card in anunturi:
            item = {}

            titlu_tag = card.find('h3') 
            if not titlu_tag:
                titlu_tag = card.find('span', {'data-cy': 'listing-item-title'})
            
            item['Titlu'] = titlu_tag.get_text(strip=True) if titlu_tag else "Titlu Necunoscut"
            
            pret_text = "N/A"
            spans = card.find_all('span')
            for span in spans:
                text = span.get_text(strip=True)
                if '€' in text:
                    pret_text = text
                    break
            item['Pret_Raw'] = pret_text
            
            link_tag = card.find('a', href=True)
            item['Link'] = "https://www.storia.ro" + link_tag['href'] if link_tag else "N/A"
            
            full_text = card.get_text(separator=' | ')
            item['Full_Description'] = full_text
                        
            data_colectata.append(item)
            
        print(f"Pagina {pagina} success. Total anunturi pana acum: {len(data_colectata)}")
        
        time.sleep(random.uniform(2, 4))
        
    except Exception as e:
        print(f"Eroare la pagina {pagina}: {e}")

df = pd.DataFrame(data_colectata)

def curata_pret(p):
    if pd.isna(p) or p == "N/A": return None
    clean = p.replace(' ', '').replace('€', '').replace(',', '.')
    try:
        return float(clean)
    except:
        return None

df['Pret_EUR'] = df['Pret_Raw'].apply(curata_pret)

print(df[['Titlu', 'Pret_EUR', 'Link']].head())

nume_fisier = "3_imobiliare_iasi_raw.csv"
df.to_csv(nume_fisier, index=False)

Pagina 1: https://www.storia.ro/ro/rezultate/vanzare/apartament/iasi/iasi?page=1 ...
Pagina 1 success. Total anunturi pana acum: 37
Pagina 2: https://www.storia.ro/ro/rezultate/vanzare/apartament/iasi/iasi?page=2 ...
Pagina 2 success. Total anunturi pana acum: 74
Pagina 3: https://www.storia.ro/ro/rezultate/vanzare/apartament/iasi/iasi?page=3 ...
Pagina 3 success. Total anunturi pana acum: 111
Pagina 4: https://www.storia.ro/ro/rezultate/vanzare/apartament/iasi/iasi?page=4 ...
Pagina 4 success. Total anunturi pana acum: 148
Pagina 5: https://www.storia.ro/ro/rezultate/vanzare/apartament/iasi/iasi?page=5 ...
Pagina 5 success. Total anunturi pana acum: 185
Pagina 6: https://www.storia.ro/ro/rezultate/vanzare/apartament/iasi/iasi?page=6 ...
Pagina 6 success. Total anunturi pana acum: 222
Pagina 7: https://www.storia.ro/ro/rezultate/vanzare/apartament/iasi/iasi?page=7 ...
Pagina 7 success. Total anunturi pana acum: 259
Pagina 8: https://www.storia.ro/ro/rezultate/vanzare/apartament/iasi/ia